In [1]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [3]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

d:\housing-regression-ml\.venv\Scripts\python.exe
3.0.4
d:\housing-regression-ml\.venv\Lib\site-packages\xgboost\__init__.py


In [4]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

d:\housing-regression-ml\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("../data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("../data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [ ]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [9]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("../mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2026/01/31 20:19:38 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2026-01-31 20:19:38,169] A new study created in memory with name: no-name-46056d29-eb05-40bd-ad09-df431eb8032e
[I 2026-01-31 20:20:02,415] Trial 0 finished with value: 74433.6470038351 and parameters: {'n_estimators': 977, 'max_depth': 4, 'learning_rate': 0.033547551007055885, 'subsample': 0.5863965601016106, 'colsample_bytree': 0.7757670104745054, 'min_child_weight': 1, 'gamma': 1.5425045272203959, 'reg_alpha': 7.054883469441499e-06, 'reg_lambda': 4.383102643816204e-07}. Best is trial 0 with value: 74433.6470038351.
[I 2026-01-31 20:20:12,671] Trial 1 finished with value: 86751.21929319987 and parameters: {'n_estimators': 567, 'max_depth': 3, 'learning_rate': 0.02412005876352703, 'subsample': 0.54091106366407, 'colsample_bytree': 0.6857906332630059, 'min_child_weight': 7, 'gamma': 2.023007031383923, 'reg_alpha': 0.03871962158179033, 'reg_lambda':

Best params: {'n_estimators': 649, 'max_depth': 9, 'learning_rate': 0.024242003950437723, 'subsample': 0.5074880057625739, 'colsample_bytree': 0.6525590377424405, 'min_child_weight': 7, 'gamma': 2.1008322981441023, 'reg_alpha': 3.4056591596990745e-07, 'reg_lambda': 1.3156846216441145e-05}


In [10]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 30882.149158005483
RMSE: 70016.04565775672
R²: 0.9621160556871814


d:\housing-regression-ml\.venv\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [20:26:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2026/01/31 20:27:12 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2026/01/31 20:27:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
